In [1]:
!pip install -q \
autodistill \
autodistill-yolov8 \
roboflow \
supervision==0.9.0 \
autodistill-grounded-sam



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.0.11 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.2 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
osmnx 1.8.1 requires shapely>=2.0, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


### In this notebook we have done image precessing and create a pipeline to train our model.

In [2]:
from autodistill.detection import CaptionOntology
from autodistill_grounded_sam import GroundedSAM

In [ ]:
# copy specific number of picture for preprocessing

import shutil
import os

def copy_images_range(source_folder, destination_folder, start_index, end_index):
    
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    files = os.listdir(source_folder)

    files.sort()

    if start_index >= len(files) or end_index >= len(files) or start_index > end_index:
        print("Invalid indices provided. Please check the range.")
        return

    
    try:
        for i in range(start_index, end_index):
            source_path = os.path.join(source_folder, files[i])
            destination_path = os.path.join(destination_folder, files[i])

            shutil.copy2(source_path, destination_path)
            
        print("Copy successful.")
    except:
        print("Copy failed.")

source_folder = '/kaggle/input/cattle-weight-detection-model-dataset-12k/www.acmeai.tech Dataset - BMGF-LivestockWeight-CV/Pixel/B3/images'
destination_folder = '/kaggle/working/images'
start_index = 2400  
end_index = 2500   

copy_images_range(source_folder, destination_folder, start_index, end_index)


In [17]:
!cd /kaggle/working/
import os
HOME = os.getcwd()
print(HOME)
IMAGE_DIR_PATH = f"{HOME}/images"
DATASET_DIR_PATH = f"{HOME}/dataset"

/kaggle/working


In [18]:
# preprocessing and labling

BOX_THRESHOLD = 0.5
TEXT_THRESHOLD = 0.70


CAPTION_ONTOLOGY = {
    "a cow": "cow",
    "a circle shape paper sticker": "sticker"
}

model = GroundedSAM(
    ontology=CaptionOntology(CAPTION_ONTOLOGY),
    box_threshold=BOX_THRESHOLD,
    text_threshold=TEXT_THRESHOLD,
)

dataset = model.label(
    input_folder=IMAGE_DIR_PATH,
    extension=".jpg",
    output_folder=DATASET_DIR_PATH)

trying to load grounding dino directly
final text_encoder_type: bert-base-uncased


Labeling /kaggle/working/images/76_s_172_F.jpg: 100%|██████████| 100/100 [03:10<00:00,  1.91s/it]


Labeled dataset created - ready for distillation.


In [19]:
!rm -r /kaggle/working/images
!rm -r /kaggle/working/dataset/images
!rm -r /kaggle/working/dataset/annotations
!cd /kaggle/working/dataset

In [21]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

zip_dir()

/kaggle/working/directory.zip

### I've made further processing on those image using Roboflow